<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Run Deltalake tables on colab we need these packages

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("DeltaTableExample") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
spark

##### Creating tables in deltalake

In [3]:
from delta.tables import *
from pyspark.sql.functions import *

In [4]:
delta_table=DeltaTable.create(spark).tableName("delta_table")\
            .addColumn("id", "INT")\
            .addColumn("name", "STRING")\
            .addColumn("age", "INT")\
            .property("description", "sample table")\
            .location("/content/spark-warehouse")\
            .execute()

Logs are created as .json files

In [5]:
!cat /content/spark-warehouse/_delta_log/00000000000000000001.json

{"add":{"path":"part-00000-db1e1212-3009-4f97-97e2-662b0f18dcfe-c000.snappy.parquet","partitionValues":{},"size":908,"modificationTime":1749990905909,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":100,\"name\":\"Raju\",\"age\":20},\"maxValues\":{\"id\":100,\"name\":\"Raju\",\"age\":20},\"nullCount\":{\"id\":0,\"name\":0,\"age\":0}}"}}
{"commitInfo":{"timestamp":1749990906052,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"908"},"engineInfo":"Apache-Spark/3.2.4 Delta-Lake/2.0.0","txnId":"85f2b42a-2157-4388-b5c0-0172463332fa"}}


after every operation json logs files are created and crc files are created, and after every 10 logs files created a check point parquet file is created.

In [6]:
spark.sql('insert into delta_table values(100,"Raju",20);')

DataFrame[]

In [7]:
spark.sql("select * from delta_table").show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
| 35|SandeepM| 24|
| 12|SandeepG| 24|
| 37| Deepdas| 23|
| 33| Sandeep| 24|
| 34|   Seenu| 34|
| 23|   sampu| 23|
| 15|   Ramuy| 22|
|244|   Reema| 45|
| 26|    Remo| 32|
|100|    Raju| 20|
|100|    Raju| 20|
|100|    Raju| 20|
| 14|     Ram| 23|
+---+--------+---+



In [8]:
spark.sql('Insert into delta_table values(12,"Sandeep",24),(14,"Ram",23);')

DataFrame[]

In [9]:
!cat /content/spark-warehouse/_delta_log/00000000000000000002.json

{"add":{"path":"part-00000-cf336223-1b19-4153-a1d7-a6346fc992ef-c000.snappy.parquet","partitionValues":{},"size":929,"modificationTime":1749991431015,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":12,\"name\":\"Sandeep\",\"age\":24},\"maxValues\":{\"id\":12,\"name\":\"Sandeep\",\"age\":24},\"nullCount\":{\"id\":0,\"name\":0,\"age\":0}}"}}
{"add":{"path":"part-00001-25140bf0-d016-43a3-a1ff-b70e924793e7-c000.snappy.parquet","partitionValues":{},"size":899,"modificationTime":1749991430999,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":14,\"name\":\"Ram\",\"age\":23},\"maxValues\":{\"id\":14,\"name\":\"Ram\",\"age\":23},\"nullCount\":{\"id\":0,\"name\":0,\"age\":0}}"}}
{"commitInfo":{"timestamp":1749991431028,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":1,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"2","numOutputRows":"2","numOutputBytes":"1828"},"engineInfo":"Apa

In [10]:
spark.sql('select * from delta_table').show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
| 35|SandeepM| 24|
| 12|SandeepG| 24|
| 12| Sandeep| 24|
| 37| Deepdas| 23|
| 33| Sandeep| 24|
| 34|   Seenu| 34|
| 23|   sampu| 23|
| 15|   Ramuy| 22|
|244|   Reema| 45|
| 26|    Remo| 32|
|100|    Raju| 20|
|100|    Raju| 20|
|100|    Raju| 20|
| 14|     Ram| 23|
| 14|     Ram| 23|
+---+--------+---+



In [11]:
spark.sql('Insert into delta_table values(10,"Sagar",24),(15,"Ramuy",22),(34,"Seenu",34),(23,"sampu",23);')
spark.sql('Insert into delta_table values(35,"SandeepM",24),(244,"Reema",45),(26,"Remo",32);')

DataFrame[]

In [12]:
spark.sql('delete from delta_table where id=10;')
spark.sql('update delta_table set name="Rajesh" where id=12;')
spark.sql('insert into delta_table values(33,"Sandeep",24);')
spark.sql('insert into delta_table values(37,"Deepdas",23);')
spark.sql('update delta_table set name="Rajesh" where id=12;')

DataFrame[]

In [13]:
spark.sql('select * from delta_table').show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
| 35|SandeepM| 24|
| 35|SandeepM| 24|
| 37| Deepdas| 23|
| 37| Deepdas| 23|
| 33| Sandeep| 24|
| 33| Sandeep| 24|
| 12|  Rajesh| 24|
| 12|  Rajesh| 24|
| 34|   Seenu| 34|
| 23|   sampu| 23|
| 34|   Seenu| 34|
| 23|   sampu| 23|
| 15|   Ramuy| 22|
| 15|   Ramuy| 22|
|244|   Reema| 45|
| 26|    Remo| 32|
|244|   Reema| 45|
| 26|    Remo| 32|
|100|    Raju| 20|
|100|    Raju| 20|
+---+--------+---+
only showing top 20 rows



In [14]:
spark.sql('update delta_table set name="SandeepG" where id=12;')

DataFrame[]

In [15]:
!ls -lrt /content/spark-warehouse/_delta_log/

total 124
-rw-r--r-- 1 root root   906 Jun 15 12:28 00000000000000000000.json
-rw-r--r-- 1 root root   725 Jun 15 12:35 00000000000000000001.json
-rw-r--r-- 1 root root  1083 Jun 15 12:43 00000000000000000002.json
-rw-r--r-- 1 root root  1083 Jun 15 12:47 00000000000000000003.json
-rw-r--r-- 1 root root  1089 Jun 15 12:47 00000000000000000004.json
-rw-r--r-- 1 root root  1076 Jun 15 12:50 00000000000000000005.json
-rw-r--r-- 1 root root  1043 Jun 15 12:50 00000000000000000006.json
-rw-r--r-- 1 root root   729 Jun 15 12:51 00000000000000000007.json
-rw-r--r-- 1 root root   729 Jun 15 12:51 00000000000000000008.json
-rw-r--r-- 1 root root  1043 Jun 15 12:51 00000000000000000009.json
-rw-r--r-- 1 root root  1047 Jun 15 12:52 00000000000000000010.json
-rw-r--r-- 1 root root 12948 Jun 15 12:52 00000000000000000010.checkpoint.parquet
-rw-r--r-- 1 root root   726 Jun 15 12:55 00000000000000000011.json
-rw-r--r-- 1 root root   726 Jun 15 12:56 00000000000000000012.json
-rw-r--r-- 1 root root  

**END OF CODE**